In [1]:
import os
from pathlib import Path
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from label_flip_revised.utils import open_csv, create_dir
from label_flip_revised.simple_nn_model import SimpleModel
from label_flip_revised.torch_utils import evaluate, train_model

In [2]:
PATH_ROOT = Path(os.getcwd()).absolute().parent
print(PATH_ROOT)

/home/lukec/workspace/label_flip_revised


In [3]:
path_cm_poison = glob(os.path.join(PATH_ROOT, 'results', 'synth_nn', '*.csv'))
print(path_cm_poison[:5])

['/home/lukec/workspace/label_flip_revised/results/synth_nn/synth_nn_poison_32.csv', '/home/lukec/workspace/label_flip_revised/results/synth_nn/synth_nn_poison_43.csv', '/home/lukec/workspace/label_flip_revised/results/synth_nn/synth_nn_poison_6.csv', '/home/lukec/workspace/label_flip_revised/results/synth_nn/synth_nn_poison_31.csv', '/home/lukec/workspace/label_flip_revised/results/synth_nn/synth_nn_poison_17.csv']


In [4]:
df_c_measure = pd.DataFrame()
for p in path_cm_poison:
    _df = pd.read_csv(p)
    df_c_measure = pd.concat([df_c_measure, _df])

print(df_c_measure.shape)

(2619, 36)


In [5]:
# Remove NA
print('# of columns before removing NA:', len(df_c_measure.columns) - 1)  # Name does not count
cols_not_na = df_c_measure.columns[df_c_measure.notna().any()].tolist()
df_c_measure = df_c_measure[cols_not_na]
print('# of columns after removing NA:', len(df_c_measure.columns) - 1)  # Name does not count
print(cols_not_na)

# of columns before removing NA: 35
# of columns after removing NA: 28
['Data', 'overlapping.F1.mean', 'overlapping.F1.sd', 'overlapping.F1v.mean', 'overlapping.F2.mean', 'overlapping.F3.mean', 'overlapping.F4.mean', 'neighborhood.N1', 'neighborhood.N2.mean', 'neighborhood.N2.sd', 'neighborhood.N3.mean', 'neighborhood.N3.sd', 'neighborhood.N4.mean', 'neighborhood.N4.sd', 'neighborhood.T1.mean', 'neighborhood.T1.sd', 'neighborhood.LSC', 'linearity.L1.mean', 'linearity.L2.mean', 'linearity.L3.mean', 'dimensionality.T2', 'dimensionality.T3', 'dimensionality.T4', 'balance.C1', 'balance.C2', 'network.Density', 'network.ClsCoef', 'network.Hubs.mean', 'network.Hubs.sd']


In [ ]:
# Rename Columns
new_cols = [
    'Data', 'F1', 'F1 SD', 'F1v', 'F2', 'F3', 'F4', 'N1', 
    'N2', 'N2 SD', 'N3 ', 'N3 SD', 'N4', 'N4 SD', 'T1', 'T1 SD', 'LSC', 
    'L1', 'L2', 'L3', 'T2', 'T3', 'T4', 'C1', 'C2', 'Density', 'ClsCoef', 
    'Hubs', 'HubsSD']
new_names_map = {df_c_measure.columns[i]:new_cols[i] for i in range(len(new_cols))}

df_c_measure = df_c_measure.rename(new_names_map, axis=1)

df_c_measure.head()

In [6]:
rates = [float(Path(d).stem.split('_')[-1]) for d in df_c_measure['Data'].to_list()]
df_c_measure['rate'] = rates

In [7]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print('Running on CPU!')

HIDDEN_LAYER = 128
LR = 0.001  # Learning rate.
MAX_EPOCHS = 400  # Number of iteration for training.
BATCH_SIZE = 128

In [8]:
def get_dataloader(X, y, batch_size=BATCH_SIZE, shuffle=True):
    dataset = TensorDataset(
        torch.from_numpy(X).type(torch.float32),
        torch.from_numpy(y).type(torch.int64))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [9]:
path_output = os.path.join(PATH_ROOT, 'results')
path_model = os.path.join(path_output, 'torch')
create_dir(path_model)

In [10]:
path_plot = os.path.join(path_output, f'synth_falfa_score.csv')

acc_train = []
acc_test = []
datanames = []

if os.path.exists(path_plot):
    df = pd.read_csv(path_plot)
    acc_train = df['train'].to_list()
    acc_test = df['test'].to_list()
    datanames = df['data'].to_list()
else:
    postfix_clean = '_clean_test.csv'
    postfix_torch = '_SimpleNN.torch'

    for i in tqdm(range(df_c_measure.shape[0])):
        dataname_poison = str(Path(df_c_measure.iloc[i]['Data']).stem)
        dataname = dataname_poison[:-len('_nn_ALFA_0.20')]
        datanames.append(dataname_poison)
        
        X_po, y_po, _ = open_csv(os.path.join(PATH_ROOT, 'data', 'synth', 'alfa_nn', f'{dataname_poison}.csv'))
        X_test, y_test, _ = open_csv(os.path.join(PATH_ROOT, 'data', 'synth', 'test', f'{dataname}{postfix_clean}'))
        n_features = X_po.shape[1]

        dataloader_poison = get_dataloader(X_po, y_po, shuffle=True)
        dataloader_test = get_dataloader(X_test, y_test, shuffle=False)

        model = SimpleModel(n_features, hidden_dim=HIDDEN_LAYER, output_dim=2).to(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.8)
        loss_fn = nn.CrossEntropyLoss()

        _path_model = os.path.join(path_model, f'{dataname_poison}{postfix_torch}')
        if os.path.exists(_path_model):
            model.load_state_dict(torch.load(_path_model, map_location=device))
        else:
            train_model(model, dataloader_poison, optimizer, loss_fn, device, MAX_EPOCHS)
            torch.save(model.state_dict(), _path_model)

        acc_po, _ = evaluate(dataloader_poison, model, loss_fn, device)
        acc_te, _ = evaluate(dataloader_test, model, loss_fn, device)
        acc_train.append(acc_po)
        acc_test.append(acc_te)

    results = {
        'data': datanames,
        'train': acc_train,
        'test': acc_test,
    }
    df = pd.DataFrame(results)
    df.to_csv(path_plot, index=False)

100%|██████████| 2619/2619 [1:05:40<00:00,  1.50s/it]


In [ ]:
figsize=(10, 4)
fontsize = 14
plt.rcParams["font.size"] = fontsize

In [ ]:
cols = [
    'F1', 'F1 SD', 'F1v', 'F2', 'F3', 'F4', 'N1', 
    'N2', 'N2 SD', 'N3 ', 'N3 SD', 'N4', 'N4 SD', 'T1', 'T1 SD', 'LSC', 
    'L1', 'L2', 'L3', 'T2', 'T3', 'T4', 'C1', 'C2', 'Density', 'ClsCoef', 
    'Hubs', 'HubsSD', 'rate']

df_c_measure = df_c_measure[cols]

cor = df_c_measure[cols].corr()
cor_rate = cor['rate']

plt.figure(figsize=figsize)
cor_rate[cor_rate.abs().sort_values(ascending=False).keys()[1:]].plot.bar()
plt.title('')
plt.ylim([-1.05, 1.05])
plt.tight_layout()
name = 'synth_cor_rate_falfa'
path_fig = os.path.join(path_output, f'{name}.pdf')
plt.savefig(path_fig, dpi=300)

path_output = os.path.join(PATH_ROOT, 'results', f'{name}.csv')
cor_rate.to_csv(path_output, index=True)